In [ ]:
import mlflow
from mlflow.tracking import MlflowClient
import os

print("🚀 Unity Catalog में मॉडल को 'Staging' में ट्रांज़िशन करने की प्रक्रिया शुरू हो रही है।")

# Unity Catalog के लिए MLflow registry URI सेट करें
try:
    if "DATABRICKS_RUNTIME_VERSION" in os.environ:
        mlflow.set_registry_uri("databricks-uc")
        print("✅ MLflow रजिस्ट्री URI को यूनिटी कैटलॉग पर सेट किया गया।")
    else:
        print("⚠️ Databricks वातावरण में नहीं चल रहा है। URI सेट करना छोड़ दिया गया है।")
        
    client = MlflowClient()
    print("✅ MLflow क्लाइंट सफलतापूर्वक इनिशियलाइज़ किया गया।")
except Exception as e:
    print(f"❌ MLflow क्लाइंट इनिशियलाइज़ेशन विफल हुआ: {e}")
    exit()

# यूनिटी कैटलॉग में आपके मॉडल का पूरा नाम
model_name = "workspace.ml.house_price_model"
staging_alias = "Staging"

# नवीनतम मॉडल संस्करण प्राप्त करने का प्रयास करें
try:
    # FIX: `order_by` के बिना मॉडल संस्करणों की खोज करें, क्योंकि यह यूनिटी कैटलॉग में समर्थित नहीं है।
    model_versions = client.search_model_versions(f"name='{model_name}'")
    
    if not model_versions:
        print(f"❌ मॉडल '{model_name}' के कोई संस्करण नहीं मिले। क्या आपने इसे प्रशिक्षित और पंजीकृत किया है?")
    else:
        # FIX: परिणामों को मैन्युअल रूप से संस्करण संख्या के आधार पर सॉर्ट करें।
        latest_version = sorted(model_versions, key=lambda mv: mv.version, reverse=True)[0]
        version_number = latest_version.version
        print(f"ℹ️ नवीनतम मॉडल संस्करण मिला: v{version_number}")

        # नवीनतम संस्करण को 'Staging' उपनाम पर सेट करें
        client.set_registered_model_alias(model_name, staging_alias, version_number)
        
        print(f"✅ मॉडल संस्करण v{version_number} को सफलतापूर्वक '{staging_alias}' उपनाम पर सेट किया गया है।")
        
except Exception as e:
    print(f"❌ मॉडल को '{staging_alias}' में ट्रांज़िशन करने में त्रुटि: {e}")
    
